In [1]:
import chardet
import pandas as pd

with open('./data/tax.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


df = pd.read_csv('./data/tax.csv', encoding=result['encoding'])

df

,Date,Title,Numac,Link FR,Link NL,Text
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...
1,1/16/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...,2020010044,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR DE\nELI - Navigatie systeem via een Euro...
2,1/16/2020,SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...,2020040052,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR DE\nELI - Navigatie systeem via een Euro...
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\n\neinde eerste woord laatste woord\nPub...
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...
...,...,...,...,...,...,...
764,4/4/2022,SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021....,2022020551,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR DE\nELI - Navigatie systeem via een Euro...
765,4/4/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONC...",2022201913,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...
766,4/4/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MA...,2022031428,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...
767,4/4/2022,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGI...,40002126,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL


In [2]:
df[df.isna().any(axis=1)]

,Date,Title,Numac,Link FR,Link NL,Text
345,11/25/2020,AUTORITE FLAMANDE\n13 NOVEMBRE 2020. - Arrêté ...,2020043738,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NaN


In [3]:
df=df.drop(df.index[345])

In [4]:
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahboubeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahboubeh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Dataset
from sklearn.datasets import fetch_20newsgroups
# Data manipulation
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 100
# Text preprocessing and modelling
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
# Visualisation
#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(style='whitegrid', context='talk')
#from wordcloud import WordCloud
#import pyLDAvis
#import pyLDAvis.sklearn
# Warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
# Stopwords
stop_words = stopwords.words("dutch")

In [6]:
print(f"Number of stop words: {len(stop_words)}")

Number of stop words: 101


In [7]:
def preprocess_text(document):
    """Preprocess document into normalised tokens."""
    # Tokenise words into alphabetic tokens with minimum length of 3
    tokeniser = RegexpTokenizer(r'[A-Za-z]{3,}')
    tokens = tokeniser.tokenize(document)
    
    # Tag words with POS tag
    pos_map = {'J': 'a', 'N': 'n', 'R': 'r', 'V': 'v'}
    pos_tags = pos_tag(tokens)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(t.lower(), pos=pos_map.get(p[0], 'v')) for t, p in pos_tags]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stop_words]
    return keywords

In [9]:
# Convert to document-term matrix
vectoriser = CountVectorizer(analyzer=preprocess_text)
example_matrix = vectoriser.fit_transform(df['Text'])
# Extract feature/term names
feature_names = vectoriser.get_feature_names()
# Inspect document-term matrix
pd.DataFrame.sparse.from_spmatrix(example_matrix, 
                                  columns=feature_names)

C:\Users\Mahboubeh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aafisc,aaj,aalst,aalter,aanbestedende,aanbestedingen,aanbevelen,aanbeveling,aanbevelingen,aanbevolen,...,zwiebeln,zwiesp,zwijndrecht,zwingend,zwingende,zwingenden,zwischen,zwischenmenschliche,zwischenzeit,zwitserland
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
765,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
766,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train = vectoriser.fit_transform(df['Text'])
# Convert sparse matrix to dataframe
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}
# Rename each column using the mapping
for col in X_train.columns:
    X_train.rename(columns={col: col_map[col]}, inplace=True)
X_train

,aafisc,aaj,aalst,aalter,aanbestedende,aanbestedingen,aanbevelen,aanbeveling,aanbevelingen,aanbevolen,...,zwiebeln,zwiesp,zwijndrecht,zwingend,zwingende,zwingenden,zwischen,zwischenmenschliche,zwischenzeit,zwitserland
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.051226,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.010882,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [36]:
# Build lda model
lda = LatentDirichletAllocation(n_components=768, random_state=0)
lda.fit(example_matrix)
# Inspect topics
def describe_topics(lda, feature_names, top_n_words=5, show_weight=False):
    """Print top n words for each topic from lda model."""
    normalised_weights = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
    for i, weights in enumerate(normalised_weights):  
        print(f"********** Topic {i+1} **********")
        if show_weight:
            feature_weights = [*zip(np.round(weights, 4), feature_names)]
            feature_weights.sort(reverse=True)
            print(feature_weights[:top_n_words], '\n')
        else:
            top_words = [feature_names[i] for i in weights.argsort()[:-top_n_words-1:-1]]
            print(top_words, '\n')
describe_topics(lda, feature_names, show_weight=True)

********** Topic 1 **********
[(0.0235, 'besluit'), (0.0168, 'woord'), (0.0164, 'afdeling'), (0.0161, 'onderwijs'), (0.0156, 'houdende')] 

********** Topic 2 **********
[(0.0395, 'besluit'), (0.0341, 'lokale'), (0.0337, 'april'), (0.0323, 'vlaamse'), (0.0292, 'meet')] 

********** Topic 3 **********
[(0.1129, 'personenbelasting'), (0.09, 'aangepast'), (0.0688, 'eigen'), (0.0678, 'gelokaliseerd'), (0.0678, 'functie')] 

********** Topic 4 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 5 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 6 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 7 **********
[(0.0524, 'uren'), (0.0485, 'studentenarbeid'), (0.0377, 'zorgsector'), (0.0377, 'woorden'), (0.034, '

[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 64 **********
[(0.0296, 'steun'), (0.0176, 'begunstigde'), (0.0169, 'besluit'), (0.0145, 'art'), (0.0131, 'minister')] 

********** Topic 65 **********
[(0.0455, 'besluit'), (0.0438, 'waalse'), (0.0426, 'gelet'), (0.0422, 'regering'), (0.0227, 'bepalingen')] 

********** Topic 66 **********
[(0.0215, 'covid'), (0.0206, 'wegens'), (0.0179, 'meet'), (0.0143, 'periode'), (0.0141, 'kader')] 

********** Topic 67 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 68 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 69 **********
[(0.0237, 'vaccinatie'), (0.0179, 'operationeel'), (0.0178, 'hulpverleningszones'), (0.0142, 'woord'), (0.0138, 'prestaties')] 

********** Topi

[(0.0311, 'artikel'), (0.0293, 'mei'), (0.0249, 'decreet'), (0.0235, 'lid'), (0.0228, 'besluit')] 

********** Topic 123 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 124 **********
[(0.0375, 'gespecialiseerde'), (0.0353, 'detailhandel'), (0.0337, 'winkels'), (0.0186, 'steun'), (0.0153, 'besluit')] 

********** Topic 125 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 126 **********
[(0.0489, 'nace'), (0.0488, 'bel'), (0.0437, 'code'), (0.0138, 'verordening'), (0.0136, 'euro')] 

********** Topic 127 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 128 **********
[(0.059, 'generaal'), (0.0416, 'mobiliteit'), (0.0399, 'adjunct'), (0.0394, 'wettelijk'), (0.0334, 'tweetalig')] 

********** Topic 129

[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 183 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 184 **********
[(0.0683, 'wet'), (0.0294, 'woord'), (0.0267, 'kamer'), (0.0257, 'volksvertegenwoordigers'), (0.0249, 'artikel')] 

********** Topic 185 **********
[(0.0401, 'des'), (0.0363, 'geb'), (0.028, 'ausk'), (0.0247, 'nfte'), (0.0219, 'udes')] 

********** Topic 186 **********
[(0.06, 'centrum'), (0.0443, 'voorzitter'), (0.0432, 'directiecomit'), (0.0413, 'besluit'), (0.0352, 'december')] 

********** Topic 187 **********
[(0.0661, 'toeristische'), (0.0617, 'toerisme'), (0.0366, 'meet'), (0.0304, 'steun'), (0.0263, 'commissariaat')] 

********** Topic 188 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 

[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 244 **********
[(0.0605, 'besluit'), (0.0425, 'taksen'), (0.0386, 'kantoor'), (0.0373, 'augustus'), (0.0366, 'artikel')] 

********** Topic 245 **********
[(0.0343, 'decreet'), (0.0326, 'minister'), (0.025, 'regering'), (0.0248, 'woord'), (0.0224, 'franse')] 

********** Topic 246 **********
[(0.2968, 'fiscale'), (0.0409, 'artikel'), (0.0393, 'schuldvorderingen'), (0.0339, 'invordering'), (0.0296, 'gedwongen')] 

********** Topic 247 **********
[(0.0244, 'overwegende'), (0.0235, 'regering'), (0.0234, 'besluit'), (0.0227, 'maart'), (0.0205, 'waalse')] 

********** Topic 248 **********
[(0.0865, 'onderneming'), (0.058, 'categorie'), (0.0445, 'minstens'), (0.0414, 'serre'), (0.0368, 'openlucht')] 

********** Topic 249 **********
[(0.0213, 'financi'), (0.0185, 'toepassing'), (0.0156, 'juli'), (0.0147, 'ter'), (0.0145, 'meet')] 

********** Topic 250 ******

...

[(0.0289, 'lid'), (0.0231, 'bedoeld'), (0.0223, 'artikel'), (0.0211, 'eerste'), (0.0208, 'wetboek')] 

********** Topic 489 **********
[(0.0617, 'brusselse'), (0.0589, 'regering'), (0.0545, 'hoofdstedelijke'), (0.0487, 'brussel'), (0.0437, 'besluit')] 

********** Topic 490 **********
[(0.0485, 'bericht'), (0.0292, 'lid'), (0.0283, 'eerste'), (0.0277, 'wanneer'), (0.0229, 'artikel')] 

********** Topic 491 **********
[(0.0544, 'und'), (0.0312, 'erlass'), (0.0293, 'niglichen'), (0.0257, 'dienstleistungen'), (0.0246, 'den')] 

********** Topic 492 **********
[(0.029, 'artikel'), (0.0253, 'wet'), (0.023, 'verkiezingen'), (0.0176, 'sociale'), (0.0166, 'procedure')] 

********** Topic 493 **********
[(0.022, 'lijn'), (0.022, 'laureaat'), (0.022, 'duitstalige'), (0.022, 'douane'), (0.022, 'controleurs')] 

********** Topic 494 **********
[(0.0347, 'besluit'), (0.0265, 'maatregelen'), (0.0245, 'toegang'), (0.0237, 'covid'), (0.0234, 'publieke')] 

********** Topic 495 **********
[(0.0447, 'un

[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 547 **********
[(0.0121, 'les'), (0.0121, 'artikel'), (0.0116, 'financi'), (0.0115, 'le'), (0.0103, 'que')] 

********** Topic 548 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 549 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 550 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 551 **********
[(0.0433, 'lid'), (0.03, 'artikel'), (0.0283, 'besluit'), (0.0223, 'sluiting'), (0.0156, 'study')] 

********** Topic 552 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 553 *

[(0.0248, 'inleiding'), (0.0248, 'dagtarieven'), (0.0248, 'besproken'), (0.0248, 'aangerekende'), (0.0211, 'kalenderjaren')] 

********** Topic 606 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 607 **********
[(0.0311, 'und'), (0.0254, 'des'), (0.0181, 'von'), (0.0161, 'vom'), (0.0156, 'artikel')] 

********** Topic 608 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 609 **********
[(0.1056, 'art'), (0.0534, 'afvalstoffen'), (0.0494, 'technische'), (0.041, 'ingraving'), (0.0392, 'verbranding')] 

********** Topic 610 **********
[(0.0274, 'betuigd'), (0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen')] 

********** Topic 611 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 



[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 666 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 667 **********
[(0.0539, 'woorden'), (0.0513, 'waalse'), (0.0445, 'overheidsdienst'), (0.0443, 'fiscaliteit'), (0.0308, 'vervangen')] 

********** Topic 668 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 669 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 670 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 671 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0

[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 727 **********
[(0.0277, 'erkende'), (0.0245, 'steun'), (0.0206, 'ondernemingen'), (0.0204, 'euro'), (0.018, 'werknemers')] 

********** Topic 728 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 729 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 730 **********
[(0.0549, 'mei'), (0.0466, 'juni'), (0.0239, 'vanaf'), (0.0201, 'mensen'), (0.0201, 'alcoholverbod')] 

********** Topic 731 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

********** Topic 732 **********
[(0.0, 'zwitserland'), (0.0, 'zwischenzeit'), (0.0, 'zwischenmenschliche'), (0.0, 'zwischen'), (0.0, 'zwingenden')] 

***

In [10]:
# Build lda model
lda = LatentDirichletAllocation(n_components=2, random_state=0)
lda.fit(example_matrix)
# Inspect topics
def describe_topics(lda, feature_names, top_n_words=5, show_weight=False):
    """Print top n words for each topic from lda model."""
    normalised_weights = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
    for i, weights in enumerate(normalised_weights):  
        print(f"********** Topic {i+1} **********")
        if show_weight:
            feature_weights = [*zip(np.round(weights, 4), feature_names)]
            feature_weights.sort(reverse=True)
            print(feature_weights[:top_n_words], '\n')
        else:
            top_words = [feature_names[i] for i in weights.argsort()[:-top_n_words-1:-1]]
            print(top_words, '\n')
describe_topics(lda, feature_names, show_weight=True)


********** Topic 1 **********
[(0.018, 'artikel'), (0.0167, 'besluit'), (0.0155, 'meet'), (0.0107, 'covid'), (0.0078, 'lid')] 

********** Topic 2 **********
[(0.0215, 'und'), (0.0157, 'artikel'), (0.0142, 'des'), (0.0105, 'von'), (0.0097, 'das')] 



In [11]:
df[['topic1', 'topic2']] = lda.transform(example_matrix)
df

,Date,Title,Numac,Link FR,Link NL,Text,topic1,topic2
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXELLES-CAPITALE\n8 JANVIER 2020. - Arrêté ministériel...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-14&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-14&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.940785,0.059215
1,1/16/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE 2017. - Arrêté royal modifiant l'arrêté royal n° ...,2020010044,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-16&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-16&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.004022,0.995978
2,1/16/2020,"SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 2018. - Arrêté royal modifiant l'AR/CIR 92, en ce qu...",2020040052,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-16&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-16&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.000543,0.999457
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEMBRE 2019. - Arrêté ministériel approuvant le dossi...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-24&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-24&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-24\nNumac : 2020010214\nMINISTER...,0.997945,0.002055
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté royal remédiant au double emploi de l...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.958603,0.041397
...,...,...,...,...,...,...,...,...
764,4/4/2022,"SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021. - Arrêté royal modifiant le tableau g, section pr...",2022020551,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.001289,0.998711
765,4/4/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONCERTATION SOCIALE\n31 MARS 2022. - Arrêté royal pro...",2022201913,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.999283,0.000717
766,4/4/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MARS 2022. - Arrêté royal modifiant l'arrêté royal n...,2022031428,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.997751,0.002249
767,4/4/2022,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGION DE BRUXELLES-CAPITALE\nSERVICE PUBLIC REGIONAL ...,40002126,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL,0.500000,0.500000


In [12]:
df['top'] = df.iloc[:, 6:8].idxmax(axis=1)
df['prop'] = df.iloc[:, 6:8].max(axis=1)
df

,Date,Title,Numac,Link FR,Link NL,Text,topic1,topic2,top,prop
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXELLES-CAPITALE\n8 JANVIER 2020. - Arrêté ministériel...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-14&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-14&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.940785,0.059215,topic1,0.940785
1,1/16/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE 2017. - Arrêté royal modifiant l'arrêté royal n° ...,2020010044,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-16&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-16&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.004022,0.995978,topic2,0.995978
2,1/16/2020,"SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 2018. - Arrêté royal modifiant l'AR/CIR 92, en ce qu...",2020040052,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-16&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-16&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.000543,0.999457,topic2,0.999457
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEMBRE 2019. - Arrêté ministériel approuvant le dossi...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-24&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-24&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-24\nNumac : 2020010214\nMINISTER...,0.997945,0.002055,topic1,0.997945
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté royal remédiant au double emploi de l...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.958603,0.041397,topic1,0.958603
...,...,...,...,...,...,...,...,...,...,...
764,4/4/2022,"SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021. - Arrêté royal modifiant le tableau g, section pr...",2022020551,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.001289,0.998711,topic2,0.998711
765,4/4/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONCERTATION SOCIALE\n31 MARS 2022. - Arrêté royal pro...",2022201913,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.999283,0.000717,topic1,0.999283
766,4/4/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MARS 2022. - Arrêté royal modifiant l'arrêté royal n...,2022031428,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.997751,0.002249,topic1,0.997751
767,4/4/2022,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGION DE BRUXELLES-CAPITALE\nSERVICE PUBLIC REGIONAL ...,40002126,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.

In [14]:
topic_mapping = {'topic1': 'tax', 'topic2': 'not tax'}
df['topic'] = df['top'].map(topic_mapping)
df

,Date,Title,Numac,Link FR,Link NL,Text,topic1,topic2,top,prop,topic
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXELLES-CAPITALE\n8 JANVIER 2020. - Arrêté ministériel...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-14&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-14&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.940785,0.059215,topic1,0.940785,tax
1,1/16/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE 2017. - Arrêté royal modifiant l'arrêté royal n° ...,2020010044,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-16&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-16&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.004022,0.995978,topic2,0.995978,not tax
2,1/16/2020,"SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 2018. - Arrêté royal modifiant l'AR/CIR 92, en ce qu...",2020040052,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-16&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-16&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.000543,0.999457,topic2,0.999457,not tax
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEMBRE 2019. - Arrêté ministériel approuvant le dossi...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-24&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-24&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-24\nNumac : 2020010214\nMINISTER...,0.997945,0.002055,topic1,0.997945,tax
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté royal remédiant au double emploi de l...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.958603,0.041397,topic1,0.958603,tax
...,...,...,...,...,...,...,...,...,...,...,...
764,4/4/2022,"SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021. - Arrêté royal modifiant le tableau g, section pr...",2022020551,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR DE\nELI - Navigatie systeem via een Europese identificatiecode voor wetgeving\nhttp://www....,0.001289,0.998711,topic2,0.998711,not tax
765,4/4/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONCERTATION SOCIALE\n31 MARS 2022. - Arrêté royal pro...",2022201913,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.999283,0.000717,topic1,0.999283,tax
766,4/4/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MARS 2022. - Arrêté royal modifiant l'arrêté royal n...,2022031428,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.997751,0.002249,topic1,0.997751,tax
767,4/4/2022,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGION DE BRUXELLES-CAPITALE\nSERVICE PUBLIC REGIONAL ...,40002126,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_da